In [10]:
import os
import xml.etree.ElementTree as ET
import pandas as pd 
import re
import nltk
import sklearn as sk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer



os.chdir('/home/shanks/business')

os.listdir()
tree=[]
for item in os.listdir():
    tree.append(ET.parse(item))

def clean_text(text):
    text = re.sub(r"\s+", " ", text)
    return text

def punctuation(string):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for x in string.lower():
        if x in punctuations:
            string =string.replace(x,"")
    return string

cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "i'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

#this Function used to expand words like you've = you have
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text.lower())

In [11]:
dfcols=['Text']
data = pd.DataFrame(columns=dfcols)
i=0
new_tfidf=[]
test_tfidf=[]
tfidf = TfidfVectorizer()
for i in range(0,len(tree)):
    xtree=tree[i]
    i=i+1
    xroot=xtree.getroot()
    f_text=xroot[1].text
    data = data.append(pd.Series(f_text, index=dfcols),ignore_index=True)

stop_words = set(stopwords.words('english'))
i=0
while i<5:
    sample_text = data.loc[i,'Text']
    sample_text.lower()
    data.loc[i,'Text'] = expandContractions(sample_text)
    sample_text = data.loc[i,'Text']
    data.loc[i,'Text'] = clean_text(sample_text)
    sample_text = data.loc[i,'Text']
    data.loc[i,'Text'] = punctuation(sample_text)
    data.loc[i,'Text'] = ' '.join([word for word in sample_text.split() if word not in stop_words])
    new_tfidf.append(data.loc[i,'Text'])
    i=i+1



def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        if val>0:
            idfDict[word] = math.log10(N / float(val))
    return idfDict

def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [12]:
sample_text = data.loc[0,'Text']
new_text=sample_text.split(" ")
i=0
while i<5:
    sample_text1 = data.loc[i,'Text']
    new_text1=sample_text1.split(" ")
    new_text = set(new_text).union(set(new_text1))
    i=i+1

wordDictA = dict.fromkeys(new_text, 0)


tfBow=[]
new_list_for_idfs=[]

for i in range(0,5):
    sample_text1 = data.loc[i,'Text']
    new_text1=sample_text1.split(" ")
    for word in new_text1:
        wordDictA[word]+=1
    tfBow.append(computeTF(wordDictA, new_text1))
    new_list_for_idfs.extend([wordDictA])
    

tfBow[2]

{'higher': 0.011627906976744186,
 'redefined': 0.005813953488372093,
 'third.': 0.0,
 'ease': 0.0,
 'funds.': 0.0,
 'government': 0.005813953488372093,
 'load': 0.0,
 'regulations': 0.0,
 'blocks': 0.011627906976744186,
 'present.': 0.005813953488372093,
 'bill': 0.005813953488372093,
 'years.': 0.005813953488372093,
 'chapter': 0.005813953488372093,
 'a.': 0.005813953488372093,
 'always': 0.005813953488372093,
 'fifth': 0.005813953488372093,
 'portfolio': 0.0,
 'combine.': 0.0,
 'mutual': 0.0,
 'two': 0.0,
 'recruiting': 0.005813953488372093,
 'ghosh,': 0.005813953488372093,
 'professionals': 0.005813953488372093,
 'requirement': 0.0,
 'job.': 0.005813953488372093,
 'gas': 0.011627906976744186,
 'help': 0.0,
 'stock': 0.005813953488372093,
 'pharmaceuticals': 0.005813953488372093,
 'promotion': 0.0,
 'biased': 0.0,
 'hectares': 0.0,
 'think': 0.0,
 '900': 0.005813953488372093,
 'deputation,': 0.005813953488372093,
 'congestion': 0.0,
 'plans,': 0.0,
 'functions': 0.005813953488372093,

In [19]:
# print(new_list_for_idfs)
print(idfs)
new_list=[]
idfs = computeIDF(new_list_for_idfs)
for j in range(0,5):
    tf=tfBow[j]
    tfidfBowA = computeTFIDF(tf, idfs)
    new_list.extend([tfidfBowA])
    master_data=pd.DataFrame(new_list)


master_data.head()

{'higher': 0.0, 'redefined': 0.0, 'third.': 0.0, 'ease': 0.0, 'funds.': 0.0, 'government': 0.0, 'load': 0.0, 'regulations': 0.0, 'blocks': 0.0, 'present.': 0.0, 'bill': 0.0, 'years.': 0.0, 'chapter': 0.0, 'a.': 0.0, 'always': 0.0, 'fifth': 0.0, 'portfolio': 0.0, 'combine.': 0.0, 'mutual': 0.0, 'two': 0.0, 'recruiting': 0.0, 'ghosh,': 0.0, 'professionals': 0.0, 'requirement': 0.0, 'job.': 0.0, 'gas': 0.0, 'help': 0.0, 'stock': 0.0, 'pharmaceuticals': 0.0, 'promotion': 0.0, 'biased': 0.0, 'hectares': 0.0, 'think': 0.0, '900': 0.0, 'deputation,': 0.0, 'congestion': 0.0, 'plans,': 0.0, 'functions': 0.0, 'told': 0.0, '8,000': 0.0, '?regulation,': 0.0, '30,000': 0.0, 'wagons': 0.0, 'ports.': 0.0, 'control': 0.0, 'b.': 0.0, 'clothing': 0.0, 'make': 0.0, 'carry': 0.0, 'away.': 0.0, 'opportunity': 0.0, 'terminal': 0.0, '20:': 0.0, '?cargo': 0.0, 'teams': 0.0, 'global': 0.0, 'mindtree': 0.0, 'visited': 0.0, 'working': 0.0, 'developments': 0.0, 'give': 0.0, 'debt-equity': 0.0, 'moderated': 0.0, '

,higher,redefined,third.,ease,funds.,government,load,regulations,blocks,present.,...,jain.,overseas,factor,seems,launched,camry.,here.,principal,officer,luxury
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
response = tfidf.fit_transform(new_tfidf)
feature_names = tfidf.get_feature_names()
for col in response.nonzero()[1]:
    print (feature_names[col], ' - ', response[0, col])

concept  -  0.048193210791855776
redefined  -  0.048193210791855776
already  -  0.048193210791855776
ino  -  0.048193210791855776
toyohiko  -  0.048193210791855776
according  -  0.0777639580354771
experience  -  0.048193210791855776
presidential  -  0.048193210791855776
drive  -  0.048193210791855776
comfortable  -  0.048193210791855776
smooth  -  0.048193210791855776
extremely  -  0.048193210791855776
incorporated  -  0.048193210791855776
control  -  0.048193210791855776
cruise  -  0.048193210791855776
features  -  0.048193210791855776
advanced  -  0.048193210791855776
premium  -  0.048193210791855776
rest  -  0.048193210791855776
higher  -  0.03888197901773855
notch  -  0.048193210791855776
brand  -  0.048193210791855776
version  -  0.09638642158371155
place  -  0.03888197901773855
safety  -  0.048193210791855776
improved  -  0.048193210791855776
well  -  0.048193210791855776
exterior  -  0.048193210791855776
interior  -  0.048193210791855776
revamped  -  0.048193210791855776
enhance

indus  -  0.0
delegation  -  0.0
came  -  0.0
prescription  -  0.0
state  -  0.0
facilitate  -  0.0
knowledge  -  0.0
beyond  -  0.0
look  -  0.0
efforts  -  0.0
fine  -  0.0
technology  -  0.0
information  -  0.0
focusing  -  0.0
24  -  0.0
dec  -  0.0
reporter  -  0.0
tie  -  0.0
recipe  -  0.0
growth  -  0.0
bengal  -  0.0
past  -  0.0
like  -  0.0
session  -  0.0
added  -  0.0
industry  -  0.0
working  -  0.0
staff  -  0.0
chairman  -  0.0
also  -  0.0
minister  -  0.0
higher  -  0.03888197901773855
new  -  0.09682666481283585
here  -  0.03227555493761195
director  -  0.11664593705321566
managing  -  0.0777639580354771
said  -  0.0688929694904842
business  -  0.022964323163494733
calcutta  -  0.022964323163494733
telegraph  -  0.022964323163494733
scenario  -  0.0
exposure  -  0.0
increase  -  0.0
age  -  0.0
comes  -  0.0
feeder  -  0.0
concerning  -  0.0
investments  -  0.0
regulations  -  0.0
when  -  0.0
overseas  -  0.0
invest  -  0.0
investors  -  0.0
allows  -  0.0
opportuni

# For Variant tf-idf functions I have used Sublinear tf scaling and Maximum tf normalization

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
arr = ['When Lou, who has become the "father of the Internet," is shot by an unknown assailant, Jacob and Nick fire up the time machine again to save their friend.', "Mia Thermopolis is now a college graduate and on her way to Genovia to take up her duties as princess. Her best friend Lilly also joins her for the summer. Mia continues her 'princess lessons'- riding horses side-saddle, archery, and other royal. But her complicated life is turned upside down once again when she not only learns that she is to take the crown as queen earlier than expected...", 'Under the direction of a ruthless instructor, a talented young drummer begins to pursue perfection at any cost, even his humanity.']

vectorizer = TfidfVectorizer(sublinear_tf=True, analyzer='word', token_pattern=r'\w{1,}',ngram_range=(1,2),min_df=0)
tfidf_matrix = vectorizer.fit_transform(new_tfidf)

print(vectorizer.vocabulary_)


{'telegraph': 1439, 'calcutta': 229, 'business': 217, 'toyota': 1499, 'sales': 1269, 'firmly': 580, 'track': 1505, 'b': 154, 'r': 1151, 'srikanth': 1370, 'kirloskar': 807, 'md': 885, 'atsushi': 130, 'toyoshima': 1496, 'launched': 824, 'automated': 140, 'variant': 1542, 'camry': 241, 'luxury': 847, 'car': 248, 'bangalore': 162, 'wednesday': 1563, 'pti': 1138, 'oct': 973, '6': 36, 'motors': 914, 'ltd': 845, 'today': 1482, 'said': 1250, 'poor': 1072, 'performance': 1035, 'monsoon': 904, 'parts': 1018, 'country': 381, 'impact': 699, 'flagship': 584, 'product': 1123, 'qualis': 1143, 'company': 317, 'aims': 73, 'sell': 1302, 'targeted': 1425, '35': 30, '000': 0, 'units': 1524, 'december': 401, 'the': 1452, 'effect': 471, 'yet': 1594, 'experienced': 526, 'market': 875, 'buoyant': 215, 'tkml': 1477, 'deputy': 415, 'managing': 867, 'director': 436, 'k': 799, 'swamy': 1405, 'told': 1486, 'reporters': 1193, 'launch': 820, 'automatic': 142, 'transmission': 1510, 'firm': 577, 's': 1234, 'would': 15

In [16]:

print(tfidf_matrix.toarray())


[[0.11134418 0.         0.07931955 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.05262197 0.05262197 0.05262197]
 [0.03796801 0.04706036 0.         ... 0.         0.         0.        ]]


In [17]:

print(vectorizer.get_feature_names())

['000', '000 327', '000 qualis', '000 units', '1', '1 000', '1 200', '10', '10 per', '10 principal', '11', '11 000', '12', '12 000', '180', '180 new', '180 vacancies', '20', '20 fund', '20 jawaharlal', '200', '200 units', '2004', '2004 swamy', '24', '24 focusing', '30', '30 000', '327', '327 present', '35', '35 000', '39', '39 000', '48', '48 200', '6', '6 toyota', '6 union', '8', '8 000', '80', '80 20', '900', '900 crore', 'a', 'a j', 'abroad', 'abroad would', 'according', 'according her', 'according tkml', 'accountants', 'accountants required', 'achievements', 'achievements attracting', 'added', 'added jain', 'added minister', 'added opportunities', 'added private', 'adding', 'adding introduction', 'addressing', 'addressing round', 'advanced', 'advanced features', 'affected', 'affected monsoon', 'age', 'age would', 'ahead', 'ahead plan', 'aims', 'aims sell', 'aiyar', 'aiyar plans', 'aiyar right', 'aiyar said', 'all', 'all time', 'allows', 'allows investors', 'alo', 'alo ghosh', 'alre

In [ ]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(test_tfidf)
transformer

transformer.transform(test_tfidf)